# 2.2 Transactions Data Cleaning

##### Description

Basic data visualization and data formatting for transactions.csv

##### Notebook Steps

1. Connect Spark
1. Input Data
1. Examine Data
1. Data Cleaning
1. Output Data

## 1. Connect Spark

In [1]:
import pyspark
sc = pyspark.SparkContext(appName="trans-clean")
sc.setLogLevel("INFO")

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## 2. Input Data

In [2]:
import os
import zipfile

with zipfile.ZipFile('../../data/1-data_acquisition/1-transactions.output.zip', 'r') as zip_ref:
    zip_ref.extractall()
    
df = sqlContext.read.csv('1-transactions.output.csv', header=True)

## 3. Examine Data

##### show()

In [3]:
df.show()

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|++6eU4LsQ3UQ20ILS...|               32|               90|            298|               298|            0|        20170131|              20170504|        0|
|++lvGPJOinuin/8es...|               41|               30|            149|               149|            1|        20150809|              20190412|        0|
|+/GXNtXWQVfKrEDqY...|               36|               30|            180|               180|            1|        20170303|              20170422|        0|
|+/w1UrZwyka4C9oNH...|               36|            

##### count()

In [4]:
df.count()

1431009

##### describe()

In [5]:
df.describe().show()

+-------+--------------------+-----------------+------------------+------------------+------------------+-------------------+--------------------+----------------------+-------------------+
|summary|                msno|payment_method_id| payment_plan_days|   plan_list_price|actual_amount_paid|      is_auto_renew|    transaction_date|membership_expire_date|          is_cancel|
+-------+--------------------+-----------------+------------------+------------------+------------------+-------------------+--------------------+----------------------+-------------------+
|  count|             1431009|          1431009|           1431009|           1431009|           1431009|            1431009|             1431009|               1431009|            1431009|
|   mean|                null|37.91835481118567| 66.01769590547649|281.78703488238017| 281.3172411913552| 0.7853025382789347|2.0168484537746444E7|   2.017110068205581E7|0.02455120827332323|
| stddev|                null|4.964804906926858|10

##### printSchema()

In [6]:
df.printSchema()

root
 |-- msno: string (nullable = true)
 |-- payment_method_id: string (nullable = true)
 |-- payment_plan_days: string (nullable = true)
 |-- plan_list_price: string (nullable = true)
 |-- actual_amount_paid: string (nullable = true)
 |-- is_auto_renew: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- membership_expire_date: string (nullable = true)
 |-- is_cancel: string (nullable = true)



##### columns

In [7]:
df.columns

['msno',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'transaction_date',
 'membership_expire_date',
 'is_cancel']

##### head(5)

In [8]:
df.head(5)

[Row(msno='++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=', payment_method_id='32', payment_plan_days='90', plan_list_price='298', actual_amount_paid='298', is_auto_renew='0', transaction_date='20170131', membership_expire_date='20170504', is_cancel='0'),
 Row(msno='++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=', payment_method_id='41', payment_plan_days='30', plan_list_price='149', actual_amount_paid='149', is_auto_renew='1', transaction_date='20150809', membership_expire_date='20190412', is_cancel='0'),
 Row(msno='+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=', payment_method_id='36', payment_plan_days='30', plan_list_price='180', actual_amount_paid='180', is_auto_renew='1', transaction_date='20170303', membership_expire_date='20170422', is_cancel='0'),
 Row(msno='+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=', payment_method_id='36', payment_plan_days='30', plan_list_price='180', actual_amount_paid='180', is_auto_renew='1', transaction_date='20170329', membership_expire_date='20170331', 

##### tail(5)

In [9]:
df.tail(5)

[Row(msno='zwF50wwaJI2TBKWhB42HRBJ6EQK0jgSo1Xmwb9Jq3SU=', payment_method_id='32', payment_plan_days='180', plan_list_price='536', actual_amount_paid='536', is_auto_renew='0', transaction_date='20170215', membership_expire_date='20170817', is_cancel='0'),
 Row(msno='zx/h5MzQQmsSat04wSfGpHp6N8aWLLwM1+7OV7ujmPY=', payment_method_id='41', payment_plan_days='30', plan_list_price='149', actual_amount_paid='149', is_auto_renew='1', transaction_date='20170306', membership_expire_date='20170406', is_cancel='0'),
 Row(msno='zxvgjIKjy18Fm+cIWUfYKr68z09+ILBxuMW0DnbeUZ8=', payment_method_id='41', payment_plan_days='30', plan_list_price='99', actual_amount_paid='99', is_auto_renew='1', transaction_date='20170308', membership_expire_date='20170408', is_cancel='0'),
 Row(msno='zzNhkExbpzmpjp9tXefiCUBtgNLgS+vZE7fFfTRDJVc=', payment_method_id='38', payment_plan_days='30', plan_list_price='149', actual_amount_paid='149', is_auto_renew='0', transaction_date='20170318', membership_expire_date='20170417', i

##### Null per Column

In [10]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+----+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|   0|                0|                0|              0|                 0|            0|               0|                     0|        0|
+----+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+



##### Value Counts

In [11]:
df.groupBy('payment_method_id').count().orderBy('count').show()

+-----------------+-----+
|payment_method_id|count|
+-----------------+-----+
|                5|    1|
|               24|    4|
|                2|    4|
|               25|    5|
|               10|   40|
|                3|   42|
|               11|   79|
|                8|  179|
|                6|  186|
|               26|  668|
|               14|  672|
|               18|  714|
|               16| 1842|
|               21| 1846|
|               27| 2074|
|               19| 2136|
|               17| 2532|
|               23| 2719|
|               12| 2858|
|               28| 3452|
+-----------------+-----+
only showing top 20 rows



In [12]:
df.groupBy('payment_plan_days').count().orderBy('count').show()

+-----------------+-----+
|payment_plan_days|count|
+-----------------+-----+
|               31|    4|
|                3|    9|
|               21|   11|
|              110|   20|
|               35|   29|
|              230|   35|
|               45|   41|
|               80|   43|
|               70|   49|
|               14|   82|
|               10|  416|
|                1|  676|
|              270|  997|
|              450| 1762|
|              400| 1817|
|                0| 2218|
|              200| 3108|
|               60| 3134|
|              415| 3298|
|              240| 3440|
+-----------------+-----+
only showing top 20 rows



In [13]:
df.groupBy('plan_list_price').count().orderBy('count').show()

+---------------+-----+
|plan_list_price|count|
+---------------+-----+
|             30|    1|
|             15|    1|
|             50|    2|
|            265|    4|
|            143|    4|
|           1300|    6|
|            105|   11|
|             70|   11|
|            131|   24|
|           1260|   25|
|              1|   25|
|           1150|   35|
|            400|   43|
|            126|   46|
|            350|   49|
|            210|   65|
|            596|   66|
|            134|  109|
|           2000|  130|
|           1399|  137|
+---------------+-----+
only showing top 20 rows



In [14]:
df.groupBy('actual_amount_paid').count().orderBy('count').show()

+------------------+-----+
|actual_amount_paid|count|
+------------------+-----+
|              1778|    1|
|               849|    1|
|                15|    1|
|               897|    1|
|                30|    1|
|              1780|    1|
|               984|    1|
|                50|    2|
|               143|    4|
|               265|    4|
|              1300|    6|
|                70|   11|
|               105|   11|
|               131|   24|
|              1260|   25|
|                 1|   25|
|              1150|   35|
|               400|   43|
|               127|   46|
|               350|   49|
+------------------+-----+
only showing top 20 rows



In [15]:
df.groupBy('is_auto_renew').count().orderBy('count').show()

+-------------+-------+
|is_auto_renew|  count|
+-------------+-------+
|            0| 307234|
|            1|1123775|
+-------------+-------+



In [16]:
df.groupBy('is_cancel').count().orderBy('count').show()

+---------+-------+
|is_cancel|  count|
+---------+-------+
|        1|  35133|
|        0|1395876|
+---------+-------+



## 4. Data Cleaning

In [17]:
from pyspark.sql import types
from pyspark.sql.functions import col, to_date

### Columns

##### msno
The msno column corresponds to user ids for the dataset, so the column is renamed from msno to user_id.

In [18]:
df = df.withColumnRenamed("msno","user_id")

##### payment_method_id
The payment_method_id column is cast from string to integer.

In [19]:
df = df.withColumn("payment_method_id",col("payment_method_id").cast(types.IntegerType()))

##### payment_plan_days
The payment_plan_days column is cast from string to integer.

In [20]:
df = df.withColumn("payment_plan_days",col("payment_plan_days").cast(types.IntegerType()))

##### plan_list_price
The plan_list_price column is cast from string to integer.

In [21]:
df = df.withColumn("plan_list_price",col("plan_list_price").cast(types.IntegerType()))

##### actual_amount_paid
The actual_amount_paid column is cast from string to integer.

In [22]:
df = df.withColumn("actual_amount_paid",col("plan_list_price").cast(types.IntegerType()))

##### is_auto_renew
The is_auto_renew column is cast from string to boolean.

In [23]:
df = df.withColumn("is_auto_renew",col("is_auto_renew").cast(types.BooleanType()))

##### transaction_date
The transaction_date column must be parsed and cast to a date object.

In [24]:
df= df.withColumn('transaction_date',to_date(df.transaction_date, 'yyyyMMdd'))

##### membership_expire_date
The membership_expire_date column must be parsed and cast to a date object.

In [25]:
df= df.withColumn('membership_expire_date',to_date(df.membership_expire_date, 'yyyyMMdd'))

##### is_cancel
The is_cancel column is cast from string to boolean.

In [26]:
df = df.withColumn("is_cancel",col("is_cancel").cast(types.BooleanType()))

## 5. Data Output

##### Final Check

In [27]:
df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- payment_method_id: integer (nullable = true)
 |-- payment_plan_days: integer (nullable = true)
 |-- plan_list_price: integer (nullable = true)
 |-- actual_amount_paid: integer (nullable = true)
 |-- is_auto_renew: boolean (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- membership_expire_date: date (nullable = true)
 |-- is_cancel: boolean (nullable = true)



In [28]:
df.show()

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|             user_id|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|++6eU4LsQ3UQ20ILS...|               32|               90|            298|               298|        false|      2017-01-31|            2017-05-04|    false|
|++lvGPJOinuin/8es...|               41|               30|            149|               149|         true|      2015-08-09|            2019-04-12|    false|
|+/GXNtXWQVfKrEDqY...|               36|               30|            180|               180|         true|      2017-03-03|            2017-04-22|    false|
|+/w1UrZwyka4C9oNH...|               36|            

##### Output to File

In [29]:
filepath = '../../data/2-data_cleaning/2-transactions.output.csv'

df.write.format('com.databricks.spark.csv').options(header='true').save(filepath)